In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    brier_score_loss, roc_auc_score, average_precision_score,
    mean_absolute_error, mean_squared_error, confusion_matrix,
    RocCurveDisplay, PrecisionRecallDisplay
)
from sklearn.calibration import calibration_curve


# ==========================================
# 1. METRICS & CONFIGURATION (Identical to previous script)
# ==========================================

def expected_calibration_error(y_true, y_prob, n_bins=10):
    y_true = np.asarray(y_true)
    y_prob = np.asarray(y_prob)
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    binids = np.digitize(y_prob, bins) - 1
    ece = 0.0
    for i in range(n_bins):
        idx = binids == i
        if np.any(idx):
            acc_in_bin = y_true[idx].mean()
            conf_in_bin = y_prob[idx].mean()
            prop_in_bin = idx.mean()
            ece += np.abs(acc_in_bin - conf_in_bin) * prop_in_bin
    return ece

def frost_metrics(df: pd.DataFrame):
    rows = []
    for h, g in df.groupby("horizon_h", sort=True):
        y, p = g["y_event"].values, g["p_event"].values
        mask = ~np.isnan(y) & ~np.isnan(p)
        y, p = y[mask], p[mask]
        g = g[mask]

        if len(y) == 0: continue

        brier = brier_score_loss(y, np.clip(p, 0, 1))
        ece   = expected_calibration_error(y, p, n_bins=10)

        if len(np.unique(y)) < 2:
            roc = np.nan
            pr   = np.nan
        else:
            try: roc = roc_auc_score(y, p)
            except: roc = np.nan
            try: pr  = average_precision_score(y, p)
            except: pr  = np.nan

        mae  = float(mean_absolute_error(g["y_temp"], g["yhat_temp"]))
        rmse = float(np.sqrt(mean_squared_error(g["y_temp"], g["yhat_temp"])))
        bias = float((g["yhat_temp"] - g["y_temp"]).mean())

        rows.append({
            "horizon_h": int(h),
            "brier": brier, "ece": ece, "roc_auc": roc, "pr_auc": pr,
            "mae": mae, "rmse": rmse, "bias": bias,
            "n_samples": int(len(g))
        })
    return pd.DataFrame(rows).sort_values("horizon_h")

CONFIG = {
    'horizon': [3, 6, 12, 24],
    'frost_threshold': 0.0,
    'train_file_path': '/content/train_set_filled_w_Mean_cleaned.csv',
    'test_file_path': '/content/test_set_filled_w_Mean_cleaned.csv',
    'CONFUSION_THRESHOLD': 0.5 # Threshold for converting probabilities to classes
}

# Tuned Hyperparameters (As determined in your previous step)
TUNED_PARAMS = {
    3: {
        'clf': {'colsample_bytree': float(0.9932923543227152), 'gamma': float(2.3338144662398994), 'learning_rate': float(0.18198808134726413), 'max_depth': 9, 'n_estimators': 70, 'subsample': float(0.7801997007878172)},
        'reg': {'alpha': float(0.05808361216819946), 'colsample_bytree': float(0.9464704583099741), 'lambda': float(0.6011150117432088), 'learning_rate': float(0.1516145155592091), 'max_depth': 8, 'n_estimators': 102, 'subsample': float(0.9879639408647978)}
    },
    6: {
        'clf': {'colsample_bytree': float(0.8447411578889518), 'gamma': float(0.6974693032602092), 'learning_rate': float(0.06842892970704363), 'max_depth': 9, 'n_estimators': 100, 'subsample': float(0.7529847965068651)},
        'reg': {'alpha': float(0.05808361216819946), 'colsample_bytree': float(0.9464704583099741), 'lambda': float(0.6011150117432088), 'learning_rate': float(0.1516145155592091), 'max_depth': 8, 'n_estimators': 102, 'subsample': float(0.9879639408647978)}
    },
    12: {
        'clf': {'colsample_bytree': float(0.8447411578889518), 'gamma': float(0.6974693032602092), 'learning_rate': float(0.06842892970704363), 'max_depth': 9, 'n_estimators': 100, 'subsample': float(0.7529847965068651)},
        'reg': {'alpha': float(0.05808361216819946), 'colsample_bytree': float(0.9464704583099741), 'lambda': float(0.6011150117432088), 'learning_rate': float(0.1516145155592091), 'max_depth': 8, 'n_estimators': 102, 'subsample': float(0.9879639408647978)}
    },
    24: {
        'clf': {'colsample_bytree': float(0.8447411578889518), 'gamma': float(0.6974693032602092), 'learning_rate': float(0.06842892970704363), 'max_depth': 9, 'n_estimators': 100, 'subsample': float(0.7529847965068651)},
        'reg': {'alpha': float(0.230893825622149), 'colsample_bytree': float(0.6964101864104046), 'lambda': float(0.6832635188254582), 'learning_rate': float(0.13199933155652419), 'max_depth': 9, 'n_estimators': 84, 'subsample': float(0.9637281608315128)}
    }
}


# ==========================================
# 2. DATA PREPARATION FUNCTIONS (Identical to previous script)
# ==========================================

def load_file(full_path):
    if not os.path.exists(full_path):
        raise FileNotFoundError(f"Could not find file at: {full_path}")
    print(f"Loading {full_path}...")
    return pd.read_csv(full_path)

def prepare_base_features(df):
    df_copy = df.copy()
    df_copy['datetime'] = pd.to_datetime(df_copy['datetime'], errors='coerce')
    df_copy = df_copy.sort_values(['station_id', 'datetime'])

    df_copy['hour_sin'] = np.sin(2 * np.pi * df_copy['datetime'].dt.hour / 24)
    df_copy['hour_cos'] = np.cos(2 * np.pi * df_copy['datetime'].dt.hour / 24)

    for lag in [1, 3, 6]:
        df_copy[f'temp_lag_{lag}'] = df_copy.groupby('station_id')['air_temp_c'].shift(lag)
        df_copy[f'dew_lag_{lag}'] = df_copy.groupby('station_id')['dew_point_c'].shift(lag)

    return df_copy.dropna()

def generate_targets(df_base, h):
    df = df_base.copy()
    df = df.sort_values(['station_id', 'datetime'])
    indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=h)

    df['y_temp'] = df.groupby('station_id')['air_temp_c'].transform(
        lambda x: x.rolling(window=indexer, min_periods=1).min()
    )
    df['y_event'] = (df['y_temp'] <= CONFIG['frost_threshold']).astype(int)
    return df.dropna()


# ==========================================
# 3. VISUALIZATION FUNCTIONS (NEW)
# ==========================================

def plot_confusion_matrix(y_true, y_pred, h):
    """Plots the confusion matrix for the classification task."""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['No Frost (0)', 'Frost (1)'],
                yticklabels=['No Frost (0)', 'Frost (1)'])
    plt.title(f'Confusion Matrix (h={h}h)')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show() # Display the plot


def plot_roc_calibration(y_true, y_prob, h):
    """Plots the ROC Curve and Calibration Curve."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # --- ROC Curve ---
    RocCurveDisplay.from_predictions(y_true, y_prob, name='XGBoost', ax=axes[0])
    axes[0].plot([0, 1], [0, 1], linestyle='--', color='red', label='Chance')
    axes[0].set_title(f'ROC Curve (h={h}h) - AUC: {roc_auc_score(y_true, y_prob):.4f}')
    axes[0].legend()

    # --- Calibration Curve (Reliability Diagram) ---
    fraction_of_positives, mean_predicted_value = calibration_curve(y_true, y_prob, n_bins=10)
    axes[1].plot(mean_predicted_value, fraction_of_positives, marker='o', label='XGBoost')
    axes[1].plot([0, 1], [0, 1], 'r--', label='Perfectly Calibrated')
    axes[1].set_xlabel('Mean Predicted Probability')
    axes[1].set_ylabel('Fraction of Positives (Actual Accuracy)')
    axes[1].set_title(f'Calibration Curve (h={h}h) - Brier: {brier_score_loss(y_true, y_prob):.4f}')
    axes[1].legend()

    plt.tight_layout()
    plt.show() # Display the plot


def plot_regression_performance(y_true, y_pred, h):
    """Plots True vs. Predicted Temperature and Residuals."""

    # Calculate residuals
    residuals = y_pred - y_true

    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # --- True vs. Predicted ---
    sns.scatterplot(x=y_true, y=y_pred, ax=axes[0], alpha=0.1, s=10)
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    axes[0].plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect Prediction')
    axes[0].set_xlabel('True Minimum Temperature (°C)')
    axes[0].set_ylabel('Predicted Minimum Temperature (°C)')
    axes[0].set_title(f'True vs. Predicted Temperature (h={h}h) - MAE: {mean_absolute_error(y_true, y_pred):.3f}')
    axes[0].legend()

    # --- Residual Distribution ---
    sns.histplot(residuals, kde=True, ax=axes[1], bins=50)
    axes[1].axvline(residuals.mean(), color='red', linestyle='--', label=f'Mean Bias: {residuals.mean():.3f}')
    axes[1].set_xlabel('Prediction Residual (Predicted - True)')
    axes[1].set_ylabel('Count')
    axes[1].set_title(f'Residual Distribution (h={h}h)')
    axes[1].legend()

    plt.tight_layout()
    plt.show() # Display the plot



# ==========================================
# 4. MAIN PIPELINE (Full Dataset Training and Visualization)
# ==========================================

def run_pipeline_xgb_full_train_and_visualize():
    """Runs the full pipeline using the TUNED XGBoost models, trains on full data, and generates plots."""

    # A. Load base data
    try:
        train_base = load_file(CONFIG['train_file_path'])
        test_base = load_file(CONFIG['test_file_path'])
    except FileNotFoundError as e:
        print(f"Error loading data: {e}")
        return None

    # B. Prepare common features
    print("Preparing base features (cyclical time, lags)...")
    train_processed_base = prepare_base_features(train_base)
    test_processed_base = prepare_base_features(test_base)

    all_summaries_xgb = []

    # Loop through each horizon
    for h_current in CONFIG['horizon']:
        print(f"\nProcessing for horizon: {h_current} hours")

        # Get the TUNED parameters for the current horizon
        params_h = TUNED_PARAMS.get(h_current)
        if not params_h:
            print(f"ERROR: No tuned parameters found for horizon h={h_current}. Skipping.")
            continue

        # Generate targets for the current horizon
        print(f"Generating targets for h={h_current}...")
        train_h_targets = generate_targets(train_processed_base, h_current)
        test_h_targets = generate_targets(test_processed_base, h_current)

        # Features
        features = ['air_temp_c', 'rel_hum_percent', 'dew_point_c', 'wind_speed_m_s',
                    'hour_sin', 'hour_cos', 'temp_lag_1', 'temp_lag_3', 'temp_lag_6']
        features = [f for f in features if f in train_h_targets.columns]
        print(f"Training with features: {features}")

        # --- XGBoost Models (TUNED PARAMETERS, FULL TRAINING DATA) ---
        print("\n--- XGBoost Models (Full Training) ---")

        # 1. Frost Classifier
        clf_params = params_h['clf'].copy()
        clf_xgb = xgb.XGBClassifier(**clf_params, n_jobs=-1, random_state=42, use_label_encoder=False, eval_metric='logloss')

        print("Training Frost Classifier (XGBoost) on FULL data...")
        clf_xgb.fit(train_h_targets[features], train_h_targets['y_event'])

        # 2. Temperature Regressor
        reg_params = params_h['reg'].copy()
        reg_xgb = xgb.XGBRegressor(**reg_params, n_jobs=-1, random_state=42)

        print("Training Temperature Regressor (XGBoost) on FULL data...")
        reg_xgb.fit(train_h_targets[features], train_h_targets['y_temp'])

        # 3. Generate predictions
        print("Generating predictions (XGBoost)...")
        y_true_clf = test_h_targets['y_event']
        y_prob_clf = clf_xgb.predict_proba(test_h_targets[features])[:, 1]
        y_pred_clf = (y_prob_clf >= CONFIG['CONFUSION_THRESHOLD']).astype(int) # Apply threshold

        y_true_reg = test_h_targets['y_temp']
        y_pred_reg = reg_xgb.predict(test_h_targets[features])

        # Store predictions for metrics
        test_h_targets['p_event_xgb'] = y_prob_clf
        test_h_targets['yhat_temp_xgb'] = y_pred_reg

        # 4. Visualization
        print(f"\n--- Generating Visualizations for h={h_current}h ---")

        print("Plotting Classification (Frost Event) Performance...")
        plot_confusion_matrix(y_true_clf, y_pred_clf, h_current)
        plot_roc_calibration(y_true_clf, y_prob_clf, h_current)

        print("Plotting Regression (Temperature) Performance...")
        plot_regression_performance(y_true_reg, y_pred_reg, h_current)

        # 5. Calculate Metrics
        results_xgb = test_h_targets.copy()
        results_xgb['p_event'] = results_xgb['p_event_xgb']
        results_xgb['yhat_temp'] = results_xgb['yhat_temp_xgb']
        results_xgb['horizon_h'] = h_current
        results_xgb = results_xgb.rename(columns={'datetime': 'timestamp'})

        print("Calculating Final Metrics (XGBoost)...")
        summary_h_xgb = frost_metrics(results_xgb)
        all_summaries_xgb.append(summary_h_xgb)

    # Final Output
    print("\n" + "="*50)
    print(f"FINAL RESULTS FOR ALL HORIZONS (XGBoost - FULL TRAIN SET)")
    print("="*50)
    final_summary_xgb = pd.concat(all_summaries_xgb).reset_index(drop=True)
    print(final_summary_xgb.to_string(index=False))

    return final_summary_xgb

if __name__ == "__main__":
    final_summary_xgb = run_pipeline_xgb_full_train_and_visualize()